<a href="https://colab.research.google.com/github/NonMagneticNeedle/reinforcement_learning/blob/master/pomdp/LSTM-PPO%20TF.KERAS/PPO_LSTM_LUNAR_LANDER_CONTINUOUS_POMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [3]:
!pip install gym[all]
!pip install box2d-py
!apt-get install python-opengl -y
!apt install xvfb -y

# Special gym environment
#!pip install gym[atari]

     |████████████████████████████████| 450kB 2.7MB/s 
     |████████████████████████████████| 122kB 9.3MB/s 
     |████████████████████████████████| 215kB 11.5MB/s 
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-49ny38ym/mujoco-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-49ny38ym/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-tp07pnm1/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the

In [4]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My Drive/ll_clean

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/ll_clean


In [0]:

import tensorflow as tf
import numpy as np
import gym
import math
import time
#import tensorflow_probability as tfp
#display gym
tf.compat.v1.disable_eager_execution()
""""""
print(tf.__version__)


class ppo():
	def __init__(self,name,s_dim,a_dim,memory,a_bound,time_steps,lstm_units):
		self.s_dim = s_dim
		self.a_dim =a_dim
		self.memory = memory
		self.a_bound =a_bound
		self.time_steps =time_steps
		self.name = name
		self.lstm_units= lstm_units
		self.actor  = self.make_actor()
		self.critic  = self.make_critic()
		
	def make_actor(self):
			

		state_inputs = tf.keras.Input(batch_shape=(1,self.time_steps,self.s_dim), name='state')
		advantage = tf.keras.Input(batch_shape=(1,self.time_steps,1 ), name="Advantage")
		action= tf.keras.Input(batch_shape=(1,self.time_steps,self.a_dim), name="action")
		x = tf.keras.layers.Masking(mask_value=-1.)(state_inputs)	
		x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(200, activation='relu'))(x)
		x,state_h, state_c = tf.keras.layers.LSTM(units= self.lstm_units,return_sequences=True,return_state=True,stateful=True,name='a_lstm')(x)

		x = tf.keras.layers.Dense(200, activation='relu')(x)
		
		mu_0 = tf.keras.layers.Dense(self.a_dim, activation='tanh')(x)
		sigma_0 = tf.keras.layers.Dense(self.a_dim, activation='relu')(x)
		mu = tf.keras.layers.Lambda(lambda x:tf.clip_by_value(x * self.a_bound,-self.a_bound ,self.a_bound) )(mu_0)
		covari = tf.keras.layers.Lambda(lambda x:tf.clip_by_value(x,1e-2 , 1e+02))(sigma_0)
		mucov=tf.keras.layers.concatenate([mu,covari],axis=-1,name = "policy_head")
		
		def proximal_policy_optimization_loss(advantage, action):
			loss_clipping = 0.2
			entropy_loss = 0.001
			pi=3.1415926
			k=self.a_dim
			action =action[0]
			advantage = advantage[0]
			def loss(y_true,y_pred):

				#print("shapes",tf.keras.backend.int_shape(y_true))
				mu =y_pred[0,:,:k]#batch,timesteps,k
				cov = y_pred[0,:,k:]#batch,timesteps,k
				#tf.print(tf.keras.backend.int_shape(mu),tf.keras.backend.int_shape(cov))
				old_mu = y_true[0,:,:k]
				old_cov = y_true[0,:,k:]

				x= action

				det = tf.keras.backend.prod(cov,axis=1,keepdims=True)
				inv = 1/cov#tf.linalg.inv(cov_mat)
				norm_const = 1.0/ ( tf.keras.backend.pow(2*pi,k/2) * tf.keras.backend.pow(det,1.0/2) )
				x_mu = x - mu
				x_mu_sq = tf.keras.backend.square(x_mu)	
				prod  = inv*x_mu_sq
				
				prod2 =tf.keras.backend.sum(prod,axis=1,keepdims=True) 
				result = tf.keras.backend.exp( -0.5 *prod2)
				pdf = norm_const * result
				
				old_det = tf.keras.backend.prod(old_cov,axis=1,keepdims=True)
				old_inv = 1/old_cov#tf.linalg.old_inv(old_cov_mat)
				old_norm_const = 1.0/ ( tf.keras.backend.pow(2*pi,k/2) * tf.keras.backend.pow(old_det,1.0/2) )
				old_x_mu = x - old_mu
				old_x_mu_sq = tf.keras.backend.square(old_x_mu)	
				old_prod  = old_inv*old_x_mu_sq
				old_prod2 =tf.keras.backend.sum(old_prod,axis=1,keepdims=True) 
				old_result = tf.keras.backend.exp( -0.5 *old_prod2)
				old_pdf = old_norm_const * old_result					
				log_pdf = tf.keras.backend.log(pdf + tf.keras.backend.epsilon())
				old_log_pdf = tf.keras.backend.log(old_pdf + tf.keras.backend.epsilon() )
				entropy =  tf.keras.backend.sum(0.5 * (tf.keras.backend.log(2. * pi * det) + 1.))
				r = tf.keras.backend.exp(log_pdf- old_log_pdf)
				loss = -tf.keras.backend.mean(tf.keras.backend.minimum(r * advantage, tf.keras.backend.clip(r, min_value=1 - loss_clipping,max_value=1 + loss_clipping) * advantage)) #+ entropy_loss *entropy
				#loss = -tf.keras.backend.minimum(r * advantage, tf.keras.backend.clip(r, min_value=1 - loss_clipping,max_value=1 + loss_clipping) * advantage) #+ entropy_loss *entropy
				
				return loss
			return loss

		actor_model= tf.keras.Model(inputs=[state_inputs, advantage,action], outputs=[mucov,state_h,state_c], name='actor_model')
		actor_model.compile(loss={'policy_head':proximal_policy_optimization_loss(advantage=advantage,action=action)} ,optimizer=tf.keras.optimizers.Adam(lr=0.00001))
		return actor_model
	def make_critic(self):
			

		state_inputs = tf.keras.Input(batch_shape=(1,self.time_steps,self.s_dim), name='state')
		#mask_value = tf.keras.Input(batch_shape=(1,self.time_steps,1 ), name="Mask_value")
		x = tf.keras.layers.Masking(mask_value=-1.)(state_inputs)	
		x = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(200, activation='relu'))(x)
		x,state_h, state_c = tf.keras.layers.LSTM(units= self.lstm_units,return_sequences=True,return_state=True,stateful=True,name='c_lstm')(x)

		x = tf.keras.layers.Dense(200, activation='relu',use_bias=True)(x)
		
		val = tf.keras.layers.Dense(1, activation=None,name = "value_head",use_bias=True)(x)
		#hope is msaking makes skips the gradients too # still it trains the layers (after lstms) bias value even if the loss is 0 due to mask
		critic_model= tf.keras.Model(inputs=state_inputs, outputs=[val,state_h,state_c], name='critic_model')
		critic_model.compile(loss={'value_head':'mse'} ,optimizer=tf.keras.optimizers.Adam(lr=0.001))
		return critic_model


	def save_weights(self):
		path=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"model.h5"
		self.model.save_weights(path)
		print("saved")
	def load_weights(self):
		path=r"C:\\Users\\Dell\\Desktop\\this_year\\SCM-RL\\ppo_continous_"+self.name+"model.h5"
		self.model.load_weights(path)
		print("loaded")
	def set_lstm_state_actor(self,h,c):
		self.actor.get_layer('a_lstm').reset_states([h,c])
	def zero_lstm_state_actor(self):
		c_0 = np.zeros((1,self.lstm_units)).astype(np.float32)
		h_0 = np.zeros((1,self.lstm_units)).astype(np.float32)
		self.actor.get_layer('a_lstm').reset_states([h_0,c_0])

	def set_lstm_state_critic(self,h,c):
		self.critic.get_layer('a_lstm').reset_states([h,c])
	def zero_lstm_state_critic(self):
		c_0 = np.zeros((1,self.lstm_units)).astype(np.float32)
		h_0 = np.zeros((1,self.lstm_units)).astype(np.float32)
		self.critic.get_layer('c_lstm').reset_states([h_0,c_0])
			
	def gae_calc(self,val,val_,rew,done):
		mask=1 
		gae=0
		gamma=0.99
		lambd = 0.95
		returns=np.zeros_like(val)
		for i in reversed(range(0,len(val))):
			mask=1
			if done[i]:
				mask = 0 	
			delta=rew[i]+gamma*val_[i]*mask - val[i]
			gae=delta+gamma*lambd*mask*gae
			returns[i]=gae+val[i]
		return returns
		
	def adv_calc(self,val,val_,rew,done):
		gamma=0.99
		returns=np.zeros_like(val)
		for i in range(0,len(val)):
			returns[i] = rew[i] + (1- done[i])*val_[i]*gamma
		return returns
	def batchify(self,lst):
		def ceil(n):
			res = int(n)
			return res if res == n or n < 0 else res+1	
		def front_pad_batch(batchsize,data):
			d = np.zeros((batchsize,data.shape[-1]))-1.
			d[batchsize-data.shape[0]:] = data
			return d	
		batchsize =self.time_steps
		full_size = ceil(lst.shape[0]/batchsize)*batchsize

		return front_pad_batch(full_size,lst)	
	def batchify_adv(self,lst):
		def ceil(n):
			res = int(n)
			return res if res == n or n < 0 else res+1	
		def front_pad_batch(batchsize,data):
			d = np.zeros((batchsize,data.shape[-1]))
			d[batchsize-data.shape[0]:] = data
			return d	
		batchsize =self.time_steps
		full_size = ceil(lst.shape[0]/batchsize)*batchsize

		return front_pad_batch(full_size,lst)			
	def batchify_musig(self,lst):
		def ceil(n):
			res = int(n)
			return res if res == n or n < 0 else res+1	
		def front_pad_batch(batchsize,data):
			d = np.zeros((batchsize,data.shape[-1]))+1
			d[batchsize-data.shape[0]:] = data
			return d	
		batchsize =self.time_steps
		full_size = ceil(lst.shape[0]/batchsize)*batchsize

		return front_pad_batch(full_size,lst)	
	def train(self,epochs=10):
		time_horizon=self.time_steps

		obs =np.array(self.memory.batch_s)
		obs_ =np.array(self.memory.batch_s_)
		hs =np.array(self.memory.batch_h)	
		cs =np.array(self.memory.batch_c)
		all_obs=self.memory.batch_s.copy()
		all_obs.append(self.memory.batch_s_[-1]  )   
		all_obs = np.array(all_obs)
		all_obs_no = all_obs.shape[0]    
		all_obs_batch = self.batchify(all_obs)               
		all_values = np.zeros((all_obs_batch.shape[0],1))
		self.zero_lstm_state_critic()    
		for b in range(0,all_obs_batch.shape[0],time_horizon):
			vb ,h,c=	self.critic.predict(x=(np.array([all_obs_batch[b:b+time_horizon]])))
			all_values[b:b+time_horizon] = vb[0]
			
		values =all_values[-all_obs_no:-1] 
		values_ = all_values[-all_obs_no+1:]	
		returns = self.gae_calc(values,values_,self.memory.batch_r,self.memory.batch_done)
		advantage=np.array(returns-values)
		Action=np.array(self.memory.batch_a)
		Old_Prediction_musig =np.array(self.memory.musig)
		Old_cov = np.array(self.memory.cov)
		#print(obs.shape,advantage.shape,Action.shape,Old_Prediction_musig.shape,returns.shape)		
		obs = self.batchify(obs)
		advantage = self.batchify_adv(advantage)
		Action =self.batchify_adv(Action)
		Old_Prediction_musig =self.batchify_musig(Old_Prediction_musig)
		returns =self.batchify(returns)
		#print(obs.shape,advantage.shape,Action.shape,Old_Prediction_musig.shape,returns.shape)
		#print(returns,obs)
		for ep in range(epochs):
			self.zero_lstm_state_actor()
			self.zero_lstm_state_critic()
			for b in range(0,obs.shape[0],time_horizon):
				#train actor
				self.actor.fit(x=(np.array([obs[b:b+time_horizon]]),np.array([advantage[b:b+time_horizon]]),np.array([Action[b:b+time_horizon]])),y={'policy_head':np.array([Old_Prediction_musig[b:b+time_horizon]])},shuffle=False, epochs=1, verbose=0)

				#train critic	
				self.critic.fit(x=(np.array([obs[b:b+time_horizon]])),y={'value_head':np.array([returns[b:b+time_horizon]])},shuffle=False, epochs=1, verbose=0)
		self.memory.clear()

	def train_crazy(self,epochs=10):
		for ep in range(epochs):
			time_horizon=self.time_steps

			obs =np.array(self.memory.batch_s)
			obs_ =np.array(self.memory.batch_s_)
			hs =np.array(self.memory.batch_h)	
			cs =np.array(self.memory.batch_c)
			all_obs=self.memory.batch_s.copy()
			all_obs.append(self.memory.batch_s_[-1]  )   
			all_obs = np.array(all_obs)
			all_obs_no = all_obs.shape[0]    
			all_obs_batch = self.batchify(all_obs)               
			all_values = np.zeros((all_obs_batch.shape[0],1))
			self.zero_lstm_state_critic()    
			for b in range(0,all_obs_batch.shape[0],time_horizon):
				vb ,h,c=	self.critic.predict(x=(np.array([all_obs_batch[b:b+time_horizon]])))
				all_values[b:b+time_horizon] = vb[0]
				
			values =all_values[-all_obs_no:-1] 
			values_ = all_values[-all_obs_no+1:]	
			returns = self.gae_calc(values,values_,self.memory.batch_r,self.memory.batch_done)
			advantage=np.array(returns-values)
			Action=np.array(self.memory.batch_a)
			Old_Prediction_musig =np.array(self.memory.musig)
			Old_cov = np.array(self.memory.cov)
			#print(obs.shape,advantage.shape,Action.shape,Old_Prediction_musig.shape,returns.shape)		
			obs = self.batchify(obs)
			advantage = self.batchify_adv(advantage)
			Action =self.batchify_adv(Action)
			Old_Prediction_musig =self.batchify_musig(Old_Prediction_musig)
			returns =self.batchify(returns)
			#print(obs.shape,advantage.shape,Action.shape,Old_Prediction_musig.shape,returns.shape)
			#print(returns,obs)
		
			self.zero_lstm_state_actor()
			self.zero_lstm_state_critic()
			for b in range(0,obs.shape[0],time_horizon):
				#train actor
				self.actor.fit(x=(np.array([obs[b:b+time_horizon]]),np.array([advantage[b:b+time_horizon]]),np.array([Action[b:b+time_horizon]])),y={'policy_head':np.array([Old_Prediction_musig[b:b+time_horizon]])},shuffle=False, epochs=1, verbose=0)

				#train critic	
				self.critic.fit(x=(np.array([obs[b:b+time_horizon]])),y={'value_head':np.array([returns[b:b+time_horizon]])},shuffle=False, epochs=1, verbose=0)
		self.memory.clear()


class Memory:
	def __init__(self):
		self.batch_s = []
		self.batch_a = []
		self.batch_r = []
		self.batch_s_ = []
		self.batch_h = []
		self.batch_c =[]
		self.batch_done = []
		self.musig = []
		self.cov=[]
	def store(self, s, a, s_, r, done,musig,cov,h,c):
		self.batch_s.append(s)
		self.batch_a.append(a)
		self.batch_r.append(r)
		self.batch_s_.append(s_)
		self.batch_done.append(done)
		self.musig.append(musig)
		self.cov.append(cov)
		self.batch_h.append(h)
		self.batch_c.append(c)
	def clear(self):
		self.batch_s.clear()
		self.batch_a.clear()
		self.batch_r.clear()
		self.batch_s_.clear()
		self.batch_done.clear()
		self.musig.clear()
		self.cov.clear()
		self.batch_h.clear()
		self.batch_c.clear()
	def cnt_samples(self):
		return len(self.batch_s)


def front_pad_single(batchsize,data):
	d = np.zeros((batchsize,data.shape[-1]))-1.
	#print("d ",d.shape ,"data",data.shape)
	d[-1] = data
	#print("padded sequence",d)
	return d




envs = ['Pendulum-v0','LunarLanderContinuous-v2','BipedalWalkerHardcore-v3']

masks=[np.array([1., 1., 0.]),np.array([1., 1., 0., 0., 1., 0., 1., 1,]),1]
env= envs[1]
mask = masks[1]#to make a mdp into pomdp
#env1=gym.make('Pendulum-v0')
env1=gym.make(env)
env1=env1.unwrapped

s_dim1 =5# env1.observation_space.shape[0]
print(s_dim1)
a_dim1 =env1.action_space.shape[0]
print(a_dim1)
a_bound1 = env1.action_space.high[0]
print(a_bound1)

memory_1=Memory()
time_horizon=64##########################

agent_1 =  ppo(name = "ppo_agent_01",s_dim=s_dim1 ,a_dim= a_dim1,memory = memory_1,a_bound=a_bound1,time_steps=time_horizon,lstm_units=200)

def mask_lander(state):
	return np.array([state[0],state[1],state[4],state[6],state[7]])




DUMMY_ACTION1, DUMMY_VALUE1 ,DUMMY_LOSS_MASK = np.zeros((1,time_horizon,a_dim1)), np.zeros((1,time_horizon, 1)),np.zeros((1,time_horizon, 1))

#check if final hidden states are consistnent oor not by taking time as 1,2,5,10 
#find vs_ of terminal state
episodes = 30000
steps = 1000
render=0
print("starting>>>")



apath="/gdrive/My Drive/ll_clean/ppo_lstm_clean_mean_normal_actormodel64.h5"
cpath="/gdrive/My Drive/ll_clean/ppo_lstm_clean_mean_normal_criticmodel64.h5"
agent_1.actor.load_weights(apath)
agent_1.critic.load_weights(cpath)
print("loaded")
avg =[]
t = time.time()
best = 0
nz = 0 
for episode in range(1,episodes):
	done1=False
	stp=0
	s1=env1.reset()
	s1 = mask_lander(s1)
	agent_1.zero_lstm_state_actor()
	
	rews1 = 0	
	if episode > 500:
		render=0

	while not done1:
		#if stp%(steps//10) ==0:
		#	print("+",end='')
		if render:
			env1.render()	
		output1= agent_1.actor.predict((np.array([front_pad_single(time_horizon,s1)]),DUMMY_VALUE1,DUMMY_ACTION1))
		#print(output1)
		out1 = output1[0][0][-1]# getting the recent action
		h = output1[1]
		c = output1[2]
		mu_pred1,cov_pred1 =out1[:a_dim1],out1[a_dim1:]
		cov_pred1 = np.diag(cov_pred1)
		action1= np.random.multivariate_normal(mu_pred1,cov_pred1,a_dim1)[0]
		a1 =np.clip(action1,-a_bound1,a_bound1 )
		s_1, reward1, done1, info1 = env1.step(a1)
		#print("s_1",s_1, end='')	
		s_1 = mask_lander(s_1)
		#print(" sub:",s_1-s1)
		stp=stp+1	


		if stp>steps:
			done1=True		
		agent_1.memory.store(s1,action1,s_1,reward1,done1,out1,cov_pred1,h,c)# s, a, s_1, r, done1,musig,h,c
		rews1+=reward1

		s1 = s_1
	# updation
	#print("|")
	if rews1>100:
		best+=1
	if rews1>0:
		nz+=1		
	avg.append(rews1)
	#print("final_state ",s_1[0],s_1[1],s_1[2],s_1[3],s_1[4])
	#print(env+" | "+str(episode)+" | "+str(rews1)+"|"+str(stp)+"| pomdpmask:"+str(mask)+" |"+"time_horizon :"+str(time_horizon))
	#print("_"*100)
	agent_1.train()
	if episode % 100 == 0:
		print("current ",episode , " average of last 100 episodes :", np.mean(np.array(avg)), " max:", np.max(np.array(avg)), " min:", np.min(np.array(avg))," best/100 :" , best, " games>0 / 100: ",nz)
		best = 0
		nz = 0
		avg.clear()

		apath="/gdrive/My Drive/ll_clean/ppo_lstm_clean_mean_normal_actormodel64.h5"
		cpath="/gdrive/My Drive/ll_clean/ppo_lstm_clean_mean_normal_criticmodel64.h5"

		#perfect solve around 700th ep
		agent_1.actor.save_weights(apath)
		agent_1.critic.save_weights(cpath)
		print(" total time taken (min): " ,(time.time()-t)/60)
		t = time.time()

		 

2.2.0


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


5
2
1.0
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
starting>>>
loaded
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
current  100  average of last 100 episodes : -19.61970705060851  max: 258.22680932846737  min: -519.7562228318736  best/100 : 24  games>0 / 100:  39
 total time taken (min):  14.987387820084889
current  200  average of last 100 episodes : -32.28575165179992  max: 261.4011588410707  min: -629.8566063411668  best/100 : 26  games>0 / 100:  37
 total time taken (min):  17.468558124701183
current  300  average of last 100 episodes : -16.585274342748697  max: 226.87161951563195  min: -427.9770642216005  best/100 : 25  games>0 / 100:  37
 total time taken (min):  17.363772869110107
current  400  average of last 100 episodes : 56.33573502067995  max: 256.58679393764885  min: -356.99964428133023  best/100 : 47  games>0 / 100:  56
 total time taken (min):  19.07840645313263

In [0]:

apath="/ppo_lstm_clean_mean_normal_actormodel64+.h5"
cpath="/ppo_lstm_clean_mean_normal_criticmodel64+.h5"

#perfect solve around 700th ep
agent_1.actor.save_weights(apath)
agent_1.critic.save_weights(cpath)
from google.colab import files
files.download(apath)
files.download(cpath)

In [3]:
#testing
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()
env1=gym.make(env)
env1=env1.unwrapped
#render = 1
for episode in range(1,2):
	done1=False
	stp=0
	s1=env1.reset()
	s1 = mask_lander(s1)
	agent_1.zero_lstm_state_actor()
	
	rews1 = 0	
	while not done1:
		if stp%(steps//5) ==0:
			print("+",end='')
		if stp%2 == 0:
			screen = env1.render(mode='rgb_array')
			plt.imshow(screen)
			plt.pause(0.001)
			ipythondisplay.clear_output(wait=True)
			#ipythondisplay.display(plt.gcf())c

		output1= agent_1.actor.predict((np.array([front_pad_single(time_horizon,s1)]),DUMMY_VALUE1,DUMMY_ACTION1))
		#print(output1)
		out1 = output1[0][0][-1]# getting the recent action
		h = output1[1]
		c = output1[2]
		mu_pred1,cov_pred1 =out1[:a_dim1],out1[a_dim1:]
		cov_pred1 = np.diag(cov_pred1)#*0
		action1= np.random.multivariate_normal(mu_pred1,cov_pred1,a_dim1)[0]
		a1 =np.clip(action1,-a_bound1,a_bound1 )
		#print(a1)
		s_1, reward1, done1, info1 = env1.step(a1)
		s_1 = mask_lander(s_1)
		stp=stp+1	
		#print("steps :",stp)
		if stp>steps:
			done1=True		
		if reward1>99:
			print("reward ", reward1," state loc ",s_1[0],s_1[1],s_1[2],s_1[3],s_1[4])	
		#agent_1.memory.store(s1,action1,val,reward1,done1,out1,cov_pred1)# s, a, s_1, r, done1,musig
		rews1+=reward1
		ipythondisplay.clear_output(wait=True)
		s1 = s_1
	# updation
	print("|")
	if episode>00:
	  print("final_state ",s_1[0],s_1[1],s_1[2],s_1[3],s_1[4])
	  print(env+" | "+str(episode)+" | "+str(rews1)+"|"+str(stp)+"| pomdpmask:"+str(mask)+" |"+"time_horizon :"+str(time_horizon))
	  print("_"*100)

|
final_state  -0.09785843 -0.00081080437 0.0022108702 1.0 0.0
LunarLanderContinuous-v2 | 1 | 145.09222705813207|900| pomdpmask:[1. 1. 0. 0. 1. 0. 1. 1.] |time_horizon :64
____________________________________________________________________________________________________


In [9]:
both left and right thrusters used at same time for better control due to lstm no rocking motion observed unlike mlp on pomdp..
when cov = 0 a stair like on off decent/ascent is observed
critic 100 times lr 
c lr = 0.001
a lr = 0.0001 at start for few 20k episodes then 0.000001 
then a lr = 0.000001   loss clip = 0.05 c lr = 0.001

SyntaxError: ignored